## This notebook is developed as part of the Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto (Module 9 Week 3)

#### Data about 'List of postal codes of Canada: M' is taken from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [1]:
# Import the libraries required
import pandas as pd
import numpy as np

# I tried using the geocoder library and the code is included it didnt work for me unfortunately. More details below.
#!pip install geocoder
#import geocoder # geocoder for the second part of the assignment

In [2]:
# Read data from the URL provided

postcodeM = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
postcodeM[0]

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
...,...,...,...
175,M5Z,Not assigned,NaN
176,M6Z,Not assigned,NaN
177,M7Z,Not assigned,NaN
178,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...


In [3]:
# Setup the final result data frame in the right format
column_names = ['Postcode', 'Borough','Neighborhood']

df_postcodeM = pd.DataFrame(columns=column_names)
df_postcodeM

,Postcode,Borough,Neighborhood


In [4]:
# Populate the database
df_postcodeM['Postcode'] = postcodeM[0]['Postal code']
df_postcodeM['Borough'] = postcodeM[0]['Borough']
df_postcodeM['Neighborhood'] = postcodeM[0]['Neighborhood']
df_postcodeM.head(5)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [5]:
# Drop rows where Borough is 'Not assigned'

indexName = df_postcodeM[df_postcodeM['Borough'] == 'Not assigned'].index
df_postcodeM.drop(indexName, inplace = True)

# replace '/' with ',' in the Neighborhood column
df_postcodeM['Neighborhood'] = df_postcodeM['Neighborhood'].str.replace("/",",", regex=True)

df_postcodeM.head(5)


,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [6]:
# Check if there are dual entries of postcode and mderge the rows
# Note: Visual inspection has confirmed that this is not the case. I am not sure why the assignment description mentions it.
#       Its possible that this was the case earlier and has since been corrected

df_postcodeM = df_postcodeM.groupby('Postcode').agg({
                                                    'Borough' : 'first',
                                                    'Neighborhood':','.join}).reset_index()

In [7]:
df_postcodeM.head(5)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
print("Having processed the data as required, the shape of the DataFrame is ", df_postcodeM.shape)

Having processed the data as required, the shape of the DataFrame is  (103, 3)


### This brings us to the end of Q1 of the Week 3 Assignment on "Segmenting and Clustering Neighborhoods in Toronto" 

In [9]:
# part 2 of the assignment requires reading Latutute and Longitute from geocoder and adding it to the data frame
# Adding columns to read Latutute and Longitute
df_postcodeM.insert(3 , 'Latitute', np.nan)
df_postcodeM.insert(4 , 'Longitute', np.nan)
df_postcodeM.head(5)

,Postcode,Borough,Neighborhood,Latitute,Longitute
0,M1B,Scarborough,"Malvern , Rouge",NaN,NaN
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",NaN,NaN
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",NaN,NaN
3,M1G,Scarborough,Woburn,NaN,NaN
4,M1H,Scarborough,Cedarbrae,NaN,NaN


#### I tried using the geocoder library with the code below but it didnt work. It didn't give any error but it just didnt give any output and I had to eventually interrupt the kernel and even that didnt work. I eventually had to close the browser window and even the kernel instance on my local laptop. 

##### I eventually used the the link provided in the assignment.

for index in df_postcodeM.index:
    lat_lng_coords = None
    postal_code = df_postcodeM['Postcode'][index]
    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng

    df_postcodeM['Latitute'] = lat_lng_coords[0]
    df_postcodeM['Longitute'] = lat_lng_coords[1]

In [10]:
lat_long = pd.read_csv('http://cocl.us/Geospatial_data')
lat_long.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
# check shape of the lat_long data frame just read, to make sure its as expected
lat_long.shape

(103, 3)

In [12]:
# compare the postcode columns and copy the corresponding Latitute and Longitute to df_postcodeM

for index in df_postcodeM.index:
    if df_postcodeM['Postcode'][index] == lat_long['Postal Code'][index]:
        df_postcodeM['Latitute'] = lat_long['Latitude']
        df_postcodeM['Longitute'] = lat_long['Longitude']

df_postcodeM.head(5)      



,Postcode,Borough,Neighborhood,Latitute,Longitute
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### This brings us to the end of Q2 of the Week 3 Assignment on "Segmenting and Clustering Neighborhoods in Toronto"